In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
import torch.nn as nn

from modules.unet import UNet_LS, UNet_LS_down, UNet_LS_up
from transfers import UNetTransfer, Transfer, pretrained_transfers
from logger import Logger, VisdomLogger
from datasets import load_train_val, load_test, load_ood, load_all
from task_configs import tasks, RealityTask
from model_configs import model_types
from models import DataParallelModel
import warnings
warnings.filterwarnings('ignore')

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_CUDA = torch.cuda.is_available()

In [56]:
PORT = 6932
SERVER = "10.90.47.7"
logger = VisdomLogger("Jupyter_vis", env="YKIVVA_jupyter_visualize_", port=PORT, server=SERVER)

Setting up a new session...


No deletion
In (git) scaling-reset
Logging to environment YKIVVA_jupyter_visualize_


# DOWNLOAD MODELS

In [149]:
MODELS_DIR = "/scratch/kivva/projects/consistency_LS/models"
MODELS_MULTITASK = "/scratch/kivva/projects/consistency_LS/multitask/results_mae/models"
MODELS_PERCEPTUAL_DIRECT = "/scratch/kivva/projects/consistency_LS/consistency/results_PERCEPTUAL_x->n|r/models"
MODELS_PERCEPTUAL_BASELINE = "/scratch/kivva/projects/consistency_LS/consistency/results_PERCEPTUAL_NORMAL_BASELINE/models"

rgb_down = model_types["rgb"]["down"][0]()
normal_down = model_types["normal"]["down"][0]()
depth_down = model_types["depth_zbuffer"]["down"][0]()
normal_up = model_types["normal"]["up"][0]()
depth_up = model_types["depth_zbuffer"]["up"][0]()

path_rgb_down = MODELS_PERCEPTUAL_DIRECT + '/rgb_down.pth'
path_normal_down = MODELS_DIR + '/normal_down.pth'
path_depth_down = MODELS_DIR + '/depth_zbuffer_down.pth'
path_normal_up = MODELS_DIR + "/normal_up.pth"
path_depth_up = MODELS_PERCEPTUAL_DIRECT + "/depth_zbuffer_up.pth"

In [150]:
if os.path.isfile(path_rgb_down):
    rgb_down.load_weights(path_rgb_down)
    print("1")
if os.path.isfile(path_normal_down):
    normal_down.load_weights(path_normal_down)
    print("2")
if os.path.isfile(path_depth_down):
    depth_down.load_weights(path_depth_down)
    print("3")
if os.path.isfile(path_normal_up):
    normal_up.load_weights(path_normal_up)
    print("4")
if os.path.isfile(path_depth_up):
    depth_up.load_weights(path_depth_up)
    print("5")

1
2
3
4
5


In [140]:
rgb2depth, path = pretrained_transfers[('rgb', 'depth_zbuffer')]
rgb2depth = DataParallelModel.load(rgb2depth(), path)

rgb2normal, path = pretrained_transfers[('rgb', 'normal')]
rgb2normal = DataParallelModel.load(rgb2normal(), path)

normal2depth, path = pretrained_transfers[('normal', 'depth_zbuffer')]
print(path)
normal2depth = DataParallelModel.load(normal2depth(), path)

depth2normal, path = pretrained_transfers[('depth_zbuffer', 'normal')]
depth2normal = DataParallelModel.load(depth2normal(), path)

/scratch/consistency_shared/models/normal2zdepth_unet_v4.pth


In [141]:
rgb2normal_down = model_types["rgb"]["down"][0]()
rgb2depth_down = model_types["rgb"]["down"][0]()
rgb2normal_up = model_types["normal"]["up"][0]()
rgb2depth_up = model_types["depth_zbuffer"]["up"][0]()

path_rgb2normal_down = MODELS_DIR + '/rgb2normal_down.pth'
path_rgb2depth_down = MODELS_DIR + '/rgb2depth_zbuffer_down.pth'
path_rgb2normal_up = MODELS_DIR + "/rgb2normal_up.pth"
path_rgb2depth_up = MODELS_DIR + "/rgb2depth_zbuffer_up.pth"

In [142]:
if os.path.isfile(path_rgb2normal_down):
    rgb2normal_down.load_weights(path_rgb2normal_down)
    print("+")
    
if os.path.isfile(path_rgb2depth_down):
    rgb2depth_down.load_weights(path_rgb2depth_down)
    print("+")

if os.path.isfile(path_rgb2normal_up):
    rgb2normal_up.load_weights(path_rgb2normal_up)
    print("+")
    
if os.path.isfile(path_rgb2depth_up):
    rgb2depth_up.load_weights(path_rgb2depth_up)
    print("+")

+
+
+
+


In [105]:
test_set = load_test(
        [tasks.rgb, tasks.normal, tasks.depth_zbuffer],
        buildings=['almena', 'albertville','espanola']
    )
test = RealityTask.from_static(
        "test", test_set, [tasks.rgb, tasks.normal, tasks.depth_zbuffer]
    )

number of images in almena:
    rgb file len: 8785
    Intersection files len:  8785
number of images in albertville:
    rgb file len: 7405
    Intersection files len:  7405
number of images in espanola:
    rgb file len: 2282
    Intersection files len:  2282


In [109]:
depth_data = test.task_data[tasks.depth_zbuffer].to(DEVICE)
normal_data = test.task_data[tasks.normal].to(DEVICE)
with torch.no_grad():
    out = model_up(model_down(depth_data))
    shape = list(normal_data.shape)
    shape[1] = 3
    out = out.clamp(min=0, max=1).expand(*shape)
    depth_data = depth_data.clamp(min=0, max=1).expand(*shape)

In [110]:
images = [normal_data, out]

logger.images_grouped(images, f"multitask_ood:rgb->normal, rgb->depth", resize=256)

# DATASET

In [87]:
tasks_set = [tasks.rgb, tasks.normal, tasks.depth_zbuffer]

buildings = ['almena']
batch_size = 64
test_loader = load_all(tasks_set, buildings, batch_size=batch_size, split_file="../data/split.txt")

    rgb file len: 8785
    Intersection files len:  8785


# ACCURACY

## DIRECT

In [152]:
src_task = tasks.depth_zbuffer
target_task = tasks.normal
tasks_set = [src_task, target_task]

buildings = ['almena']
batch_size = 64
test_loader = load_all(tasks_set, buildings, batch_size=batch_size)

test_iter = iter(test_loader)
model_down = depth_down.to(DEVICE)
model_up = normal_up.to(DEVICE)
model = depth2normal.to(DEVICE)

accuracy = 0
for in_data, out_data in test_iter:
    in_data = in_data.to(DEVICE)
    out_data = out_data.to(DEVICE)
    
    with torch.no_grad():
#         out = model(in_data)
        out = model_up(model_down(in_data))
    accuracy_tmp, _ = target_task.norm(out, out_data, compute_mse=False, batch_mean=False)
    accuracy += accuracy_tmp.sum()

print(accuracy / 8785. * 100.)

    depth_zbuffer file len: 8785
    Intersection files len:  8785
tensor(3.8416, device='cuda:0')


## PERCEPTUAL

In [146]:
src_task = tasks.rgb
middle_task = tasks.depth_zbuffer
target_task = tasks.normal
tasks_set = [src_task, target_task]

buildings = ['almena']
batch_size = 64
test_loader = load_all(tasks_set, buildings, batch_size=batch_size)

test_iter = iter(test_loader)

model_down = rgb_down.to(DEVICE)
model_up = depth_up.to(DEVICE)
model_down_per = depth_down.to(DEVICE)
model_up_per = normal_up.to(DEVICE)
model_percep = depth2normal.to(DEVICE)

accuracy = 0
for in_data, out_data in test_iter:
    in_data = in_data.to(DEVICE)
    out_data = out_data.to(DEVICE)
    
    with torch.no_grad():
#         out = model_percep(model_up(model_down(in_data)))
        out = model_up_per(model_down_per(model_up(model_down(in_data))))
    accuracy_tmp, _ = target_task.norm(out, out_data, compute_mse=False, batch_mean=False)
    accuracy += accuracy_tmp.sum()

print(accuracy / 8785. * 100.)

    rgb file len: 8785
    Intersection files len:  8785
tensor(9.3013, device='cuda:0')


# MULTITASK APPROACH

In [3]:
MULTITASK_MODELS_DIR = "/scratch/kivva/projects/consistency_LS/multitask/results_mae/models"

dest_tasks = [tasks.normal, tasks.depth_zbuffer]
src_task = tasks.rgb

model_rgb_down = model_types["rgb"]["down"][0]()
model_normal_up = model_types["normal"]["up"][0]()
model_depth_up = model_types["depth_zbuffer"]["up"][0]()

In [4]:
path_rgb_down = MULTITASK_MODELS_DIR + '/rgb_down.pth'
path_normal_up = MULTITASK_MODELS_DIR + "/normal_up.pth"
path_depth_up = MULTITASK_MODELS_DIR + "/depth_zbuffer_up.pth"

model_rgb_down.load_weights(path_rgb_down)
model_normal_up.load_weights(path_normal_up)
model_depth_up.load_weights(path_depth_up)

In [5]:
rgb2normal = UNet_LS(model_up=model_normal_up, model_down=model_rgb_down)
rgb2depth = UNet_LS(model_up=model_depth_up, model_down=model_rgb_down)

In [6]:
ood_set = load_ood([tasks.rgb])
ood = RealityTask.from_static("ood",  ood_set, [tasks.rgb])

number of ood images:  21


In [7]:
logger = VisdomLogger("visualize", env="visualize_models")

Setting up a new session...


No deletion
In (git) scaling-reset
Logging to environment visualize_models


In [8]:
input_data = ood.task_data[tasks.rgb].to(DEVICE)

In [9]:
rgb2normal.to(DEVICE)
rgb2depth.to(DEVICE)
rgb2normal = nn.DataParallel(rgb2normal) if not isinstance(rgb2normal, nn.DataParallel) else rgb2normal
rgb2depth = nn.DataParallel(rgb2depth)if not isinstance(rgb2depth, nn.DataParallel) else rgb2depth

In [10]:
with torch.no_grad():
    input_data = input_data.to(DEVICE)
    out_normal = rgb2normal(input_data)
    input_data = input_data.to(DEVICE)
    out_depth = rgb2depth(input_data)
    
    shape = list(out_normal.shape)
    shape[1] = 3
    out_normal = out_normal.clamp(min=0, max=1).expand(*shape)
    out_depth = out_depth.clamp(min=0, max=1).expand(*shape)
    

In [11]:
images = [input_data, out_normal, out_depth]

logger.images_grouped(images, f"multitask_ood:rgb->normal, rgb->depth", resize=256)

# BASELINE APPROACH

In [15]:
BASELINE_MODELS_DIR = "./baseline/models"

dest_tasks = [tasks.normal, tasks.depth_zbuffer]
src_task = tasks.rgb

baseline_rgb2normal, path_rgb2normal = pretrained_transfers[(src_task.name, dest_tasks[0].name)]
baseline_rgb2depth, path_rgb2depth = pretrained_transfers[(src_task.name, dest_tasks[1].name)]
path_rgb2normal = BASELINE_MODELS_DIR + "/unet_baseline_standardval.pth"
path_rgb2depth = BASELINE_MODELS_DIR + "/rgb2zdepth_buffer.pth"
baseline_rgb2depth = baseline_rgb2depth()
baseline_rgb2normal = baseline_rgb2normal()
baseline_rgb2depth = DataParallelModel(baseline_rgb2depth)
baseline_rgb2normal = DataParallelModel(baseline_rgb2normal)

In [15]:
baseline_rgb2depth.load_weights(path_rgb2depth)
baseline_rgb2normal.load_weights(path_rgb2normal)

In [16]:
baseline_rgb2depth.to(DEVICE)
baseline_rgb2normal.to(DEVICE)
input_data = ood.task_data[tasks.rgb].to(DEVICE)

In [17]:
with torch.no_grad():
    out_normal = baseline_rgb2normal(input_data)
    input_data.to(DEVICE)
    out_depth = baseline_rgb2depth(input_data)
    
    shape = list(out_normal.shape)
    shape[1] = 3
    out_normal = out_normal.clamp(min=0, max=1).expand(*shape)
    out_depth = out_depth.clamp(min=0, max=1).expand(*shape)

In [18]:
images = [input_data, out_normal, out_depth]

logger.images_grouped(images, f"baseline_ood:rgb->normal, rgb->depth", resize=256)

# VISUALIZE ANOTHER MODELS

In [12]:
MODELS_DIR = "/home/kivva/Consistency_LD/multitask/results/results_Multitask:rgb->noraml|depth_zbuffer/models"

dest_tasks = [tasks.normal, tasks.depth_zbuffer]
src_task = tasks.rgb

model1_rgb_down = model_types["rgb"]["down"][0]()
model1_normal_up = model_types["normal"]["up"][0]()
model1_depth_up = model_types["depth_zbuffer"]["up"][0]()

In [13]:
path_rgb_down = MULTITASK_MODELS_DIR + '/rgb_down.pth'
path_normal_up = MULTITASK_MODELS_DIR + "/normal_up.pth"
path_depth_up = MULTITASK_MODELS_DIR + "/depth_zbuffer_up.pth"

model1_rgb_down.load_weights(path_rgb_down)
model1_normal_up.load_weights(path_normal_up)
model1_depth_up.load_weights(path_depth_up)

In [14]:
rgb2normal = UNet_LS(model_up=model1_normal_up, model_down=model1_rgb_down)
rgb2depth = UNet_LS(model_up=model1_depth_up, model_down=model1_rgb_down)

In [15]:
input_data = ood.task_data[tasks.rgb].to(DEVICE)

In [16]:
rgb2normal.to(DEVICE)
rgb2depth.to(DEVICE)
rgb2normal = nn.DataParallel(rgb2normal) if not isinstance(rgb2normal, nn.DataParallel) else rgb2normal
rgb2depth = nn.DataParallel(rgb2depth)if not isinstance(rgb2depth, nn.DataParallel) else rgb2depth

In [17]:
with torch.no_grad():
    input_data = input_data.to(DEVICE)
    out_normal = rgb2normal(input_data)
    input_data = input_data.to(DEVICE)
    out_depth = rgb2depth(input_data)
    
    shape = list(out_normal.shape)
    shape[1] = 3
    out_normal = out_normal.clamp(min=0, max=1).expand(*shape)
    out_depth = out_depth.clamp(min=0, max=1).expand(*shape)

In [18]:
images = [input_data, out_normal, out_depth]

logger.images_grouped(images, f"model_ood:rgb->normal, rgb->depth", resize=256)